In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
import sys
import cv2
import matplotlib
from subprocess import check_output

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.utils import np_utils
from keras import regularizers


Using TensorFlow backend.


In [2]:
idpath='/home/shakeel/Desktop/1000IDs.txt'
lblpath='/home/shakeel/Desktop/1000labels.txt'

In [3]:
id=[]
with open(idpath) as f:
    id=f.read().split('\n')
idlbl=[]
with open(lblpath) as f:
    idlbl=f.read().split('\n')

In [4]:
dpath='/home/shakeel/Desktop/dataset1000'

In [5]:
dflz=os.listdir(dpath)

In [6]:
fullid=[]
for i in dflz:
    fullid.append(os.path.join(dpath,i))

In [7]:
from numpy import newaxis 
def converttoarray(x,y):
    feat,lbl=np.empty((0,500,500,3)),np.empty(0)
    for i,j in zip(x,y):
        img=cv2.imread(i)
        img=img[newaxis,:,:,:]
        feat=np.vstack([feat,img])
        lbl=np.append(lbl,j)
    return feat,lbl

In [8]:
x,y=converttoarray(fullid,idlbl)
onehot=pd.get_dummies(y)
target_labels= onehot.columns
y= onehot.as_matrix()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [ ]:
np.savez('/home/shakeel/Desktop/npies/d1.zpy',x)
np.savez('/home/shakeel/Desktop/l1.npy',y)

In [12]:
num_classes=5


In [13]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [14]:
X_train.shape

(750, 500, 500, 3)

In [19]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.utils import to_categorical
np.random.seed(1000)

model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(500,500,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(5))
model.add(Activation('softmax'))

#Checking the model summary
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 123, 123, 96)      34944     
_________________________________________________________________
activation_10 (Activation)   (None, 123, 123, 96)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 61, 96)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 61, 61, 96)        384       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 51, 51, 256)       2973952   
_________________________________________________________________
activation_11 (Activation)   (None, 51, 51, 256)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 25, 25, 256)       0         
__________

In [20]:
 # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  

In [ ]:
# build the model
model = baseline_model()

In [21]:
# Fit the model
#The model is fit over 3 epochs with updates every 200 images. The test data is used as the validation dataset
model.fit(X_train, y_train, epochs=3, batch_size=15, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
750/750 [==============================] - 178s 238ms/step - loss: 2.1622 - acc: 0.3093
Epoch 2/3
750/750 [==============================] - 174s 233ms/step - loss: 1.3327 - acc: 0.5573
Epoch 3/3
750/750 [==============================] - 176s 234ms/step - loss: 1.2421 - acc: 0.6187


In [22]:

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

250/250 [==============================] - 15s 62ms/step
Baseline Error: 23.20%
